In [ ]:
-- Create database
CREATE DATABASE OrderManagement;

In [2]:
-- Create tables
-- Customers Table
CREATE TABLE Customers (
    CustomerID SERIAL PRIMARY KEY,
    CustomerName VARCHAR(255) NOT NULL,
    Phone VARCHAR(50),
    AddressLine1 VARCHAR(255),
    AddressLine2 VARCHAR(255),
    City VARCHAR(50),
    State VARCHAR(50),
    PostalCode VARCHAR(50),
    Country VARCHAR(50),
    Territory VARCHAR(50),
    ContactLastName VARCHAR(100),
    ContactFirstName VARCHAR(100)
);

-- Products Table
CREATE TABLE Products (
    ProductCode VARCHAR(50) PRIMARY KEY,
    ProductLine VARCHAR(255),
    MSRP DECIMAL(10, 2)
);

-- Orders Table
CREATE TABLE Orders (
    OrderNumber INT PRIMARY KEY,
    OrderDate DATE NOT NULL,
    Status VARCHAR(50),
    QTR_ID INT,
    MONTH_ID INT,
    YEAR_ID INT,
    DealSize VARCHAR(50),
    CustomerID INT,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

-- OrderDetails Table
CREATE TABLE OrderDetails (
    OrderDetailID SERIAL PRIMARY KEY,
    OrderNumber INT,
    ProductCode VARCHAR(50),
    QuantityOrdered INT,
    PriceEach DECIMAL(10, 2),
    OrderLineNumber INT,
    Sales DECIMAL(10, 2),
    FOREIGN KEY (OrderNumber) REFERENCES Orders(OrderNumber),
    FOREIGN KEY (ProductCode) REFERENCES Products(ProductCode)
);

Commands completed successfully

Commands completed successfully

: relation "orders" already exists

Total execution time: 00:00:00.036

In [10]:
import pandas as pd
# Read CSV file 
df = pd.read_csv('/Users/meganwagnersmith/Desktop/OrderManagementData/sales_data_sample.csv', encoding='ISO-8859-1')
print(df.columns)

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID',
       'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
       'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEALSIZE'],
      dtype='object')


In [11]:
# Extracting unique customers
customers_df = df[['CUSTOMERNAME', 'CONTACTFIRSTNAME', 'CONTACTLASTNAME', 'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'POSTALCODE', 'COUNTRY']].drop_duplicates()
customers_df['CustomerID'] = range(1, 1+len(customers_df))
customers_df['ContactName'] = customers_df['CONTACTFIRSTNAME'] + ' ' + customers_df['CONTACTLASTNAME']
customers_df = customers_df[['CustomerID', 'CUSTOMERNAME', 'ContactName', 'ADDRESSLINE1', 'CITY', 'POSTALCODE', 'COUNTRY']]

# Extracting orders
orders_df = df[['ORDERNUMBER', 'ORDERDATE', 'STATUS', 'CUSTOMERNAME']].drop_duplicates()
orders_df = orders_df.merge(customers_df[['CUSTOMERNAME', 'CustomerID']], on='CUSTOMERNAME', how='left')
orders_df = orders_df[['ORDERNUMBER', 'CustomerID', 'ORDERDATE', 'STATUS']]

# Extracting order details
order_details_df = df[['ORDERNUMBER', 'PRODUCTCODE', 'QUANTITYORDERED', 'PRICEEACH']].copy()
order_details_df['OrderDetailID'] = range(1, 1+len(order_details_df))

# Extracting products 
products_df = df[['PRODUCTCODE', 'PRODUCTLINE', 'MSRP']].copy()

print(customers_df.head())
print(orders_df.head())
print(order_details_df.head())
print(products_df.head())

   CustomerID              CUSTOMERNAME      ContactName  \
0           1         Land of Toys Inc.          Kwai Yu   
1           2        Reims Collectables     Paul Henriot   
2           3           Lyon Souveniers  Daniel Da Cunha   
3           4         Toys4GrownUps.com      Julie Young   
4           5  Corporate Gift Ideas Co.      Julie Brown   

                    ADDRESSLINE1           CITY POSTALCODE COUNTRY  
0        897 Long Airport Avenue            NYC      10022     USA  
1             59 rue de l'Abbaye          Reims      51100  France  
2  27 rue du Colonel Pierre Avia          Paris      75508  France  
3             78934 Hillside Dr.       Pasadena      90003     USA  
4                7734 Strong St.  San Francisco        NaN     USA  
   ORDERNUMBER  CustomerID        ORDERDATE   STATUS
0        10107           1   2/24/2003 0:00  Shipped
1        10121           2    5/7/2003 0:00  Shipped
2        10134           3    7/1/2003 0:00  Shipped
3        1014

In [24]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

load_dotenv()
API_KEY = os.getenv("API_KEY")

# Replace these values with your database credentials
username = os.getenv('MY_USERNAME')
password = os.getenv('MY_PASSWORD')
host = 'localhost'
port = '5432'
database = 'ordermanagement'

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

In [25]:
# Insert data into customers table
customers_df.to_sql('customers', engine, if_exists='append', index=False)

# Insert data into orders table
orders_df.to_sql('orders', engine, if_exists='append', index=False)

# Insert data into order details table
order_details_df.to_sql('order_details', engine, if_exists='append', index=False)

# Insert data into products table
products_df.to_sql('products', engine, if_exists='append', index=False)


823